In [1]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv, read_excel, concat, merge_asof, DataFrame, to_datetime

In [2]:
def turnDaily(stock, info):
    daily = []
    colLabel = info.columns[1]
    i=len(info)-1
    j=len(stock)-1
    while j > -1 and i > -1:
        if info['DATE'][i] < stock['Date'][j]:
            daily.append(info[colLabel][i])
            j = j-1
        else:
            i = i-1
    return daily[::-1]

In [3]:
print('Loading Macro Data...')
unh = pd.read_csv('Data/Stocks/UNH_data.csv')
unh['Date'] = pd.to_datetime(unh['Date'])

libor = pd.read_csv('Data/liborfinal.csv')
gdp = pd.read_csv('Data/GDPC1.csv')
cpi = pd.read_csv('Data/CPIAUCSL.csv')
inflation = pd.read_csv('Data/MICH.csv')
unemployment = pd.read_csv('Data/UNRATENSA.csv')

cpi['DATE'] = pd.to_datetime(cpi['DATE'])
cpi['DATE'] = cpi['DATE'].dt.normalize()


gdp['DATE'] = pd.to_datetime(gdp['DATE'])
gdp['DATE'] = gdp['DATE'].dt.normalize()

inflation['DATE'] = pd.to_datetime(inflation['DATE'])
inflation['DATE'] = inflation['DATE'].dt.normalize()


unemployment['DATE'] = pd.to_datetime(unemployment['DATE'])
unemployment['DATE'] = unemployment['DATE'].dt.normalize()

gdpDaily = turnDaily(unh, gdp)
gdpDaily = DataFrame(gdpDaily)
gdpDaily.dropna()
gdpDaily.rename(columns = {'0':'GDP'}, inplace = True)

cpiDaily = turnDaily(unh, cpi)
cpiDaily = DataFrame(cpiDaily)
cpiDaily.dropna()
cpiDaily.rename(columns = {'0':'CPI'}, inplace = True)

inflationDaily = turnDaily(unh, inflation)
inflationDaily = DataFrame(inflationDaily)
inflationDaily.dropna()
inflationDaily.rename(columns = {'0':'Inflation'}, inplace = True)

unemploymentDaily = turnDaily(unh, unemployment)
unemploymentDaily = DataFrame(unemploymentDaily)
unemploymentDaily.dropna()
unemploymentDaily.rename(columns = {'0':'Unempl'}, inplace = True)

libor = read_csv('Data/liborfinal.csv')
libor['DATE'] = pd.to_datetime(libor['DATE'])
libor['DATE'] = libor['DATE'].dt.normalize()

Loading Macro Data...


/Users/blakehillier/anaconda3/envs/bigData/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
print('Loading Stock Data...')
#Read in files
s_and_p = ['MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AMD','AAP','AES','AET',
        'AMG','AFL','A','APD','AKAM','ALK','ALB','ARE','ALXN','ALGN','ALLE',
        'AGN','ADS','LNT','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP',
        'AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','ADI','ANDV',
        'ANSS','ANTM','AON','AOS','APA','AIV','AAPL','AMAT','APTV','ADM',
        'AJG','AIZ','T','ADSK','ADP','AZO','AVB','AVY','BLL','BAC','BK',
        'BAX','BDX','BBY','BIIB','BLK','HRB','BA','BWA','BXP','BSX'
        ,'BMY','AVGO','CHRW','CA','COG','CDNS','CPB','COF','CAH','CBOE',
        'KMX','CCL','CAT','CBS','CNC','CNP','CTL','CERN','CF','SCHW',
        'CHTR','CHK','CVX','CMG','CB','CHD','CI','XEC','CINF','CTAS','CSCO','C','CFG',
        'CTXS','CLX','CME','CMS','KO','CTSH','CL','CMCSA','CMA','CAG','CXO','COP',
        'ED','STZ','COO','GLW','COST','COTY','CCI','CSX','CMI','CVS','DHI',
        'DHR','DRI','DVA','DE','DAL','XRAY','DVN','DLR','DFS','DISCA','DISCK','DISH',
        'DG','DLTR','D','DOV','DTE','DRE','DUK','DXC','ETFC','EMN','ETN',
        'EBAY','ECL','EIX','EW','EA','EMR','ETR','EVHC','EOG','EQT','EFX','EQIX','EQR',
        'ESS','EL','ES','RE','EXC','EXPE','EXPD','ESRX','EXR','XOM','FFIV','FB','FAST',
        'FRT','FDX','FIS','FITB','FE','FISV','FLIR','FLS','FLR','FMC','FL','F',
		'FBHS','BEN','FCX','GPS','GRMN','IT','GD','GE','GIS','GM','GPC','GILD',
		'GPN','GS','GT','GWW','HAL','HBI','HOG','HIG','HAS','HCA','HP','HSIC',
		'HSY','HES','HLT','HOLX','HD','HON','HRL','HST','HPQ','HUM','HBAN','HII',
		'IDXX','INFO','ITW','ILMN','INTC','ICE','IBM','INCY','IP','IPG','IFF','INTU',
		'ISRG','IVZ','IQV','IRM','JEC','JBHT','SJM','JNJ','JCI','JPM','JNPR','KSU','K','KEY',
		'KMB','KIM','KMI','KLAC','KSS','KR','LB','LH','LRCX','LEG','LEN',
		'LLY','LNC','LKQ','LMT','L','LOW','LYB','MTB','MAC','M','MRO','MPC','MAR','MMC','MLM',
		'MAS','MA','MAT','MKC','MCD','MCK','MDT','MRK','MET','MTD','MGM','MCHP','MU',
		'MSFT','MAA','MHK','TAP','MDLZ','MON','MNST','MCO','MS','MOS','MSI','MYL','NDAQ',
		'NOV','NAVI','NTAP','NFLX','NWL','NFX','NEM','NWSA','NWS','NEE','NLSN','NKE','NI',
		'NBL','JWN','NSC','NTRS','NOC','NCLH','NRG','NUE','NVDA','ORLY','OXY','OMC','OKE',
		'ORCL','PCAR','PKG','PH','PDCO','PAYX','PNR','PBCT','PEP','PKI','PRGO','PFE',
		'PCG','PM','PSX','PNW','PXD','PNC','RL','PPG','PPL','PFG','PG','PGR',
		'PLD','PRU','PEG','PSA','PHM','PVH','PWR','QCOM','DGX','RRC','RJF','RTN','O',
		'REG','REGN','RF','RSG','RMD','RHI','ROK','COL','ROP','ROST','RCL','CRM','SBAC',
		'SLB','SNI','STX','SEE','SRE','SHW','SIG','SPG','SWKS','SLG','SNA','SO','LUV',
		'SPGI','SWK','SBUX','STT','SRCL','SYK','STI','SYF','SNPS','SYY','TROW','TPR',
		'TGT','TEL','FTI','TXN','TXT','TMO','TIF','TWX','TJX','TSCO','TDG','TRV',
		'TRIP','TSN','UDR','ULTA','USB','UAA','UNP','UAL','UNH','UPS','URI',
		'UTX','UHS','UNM','VFC','VLO','VAR','VTR','VRSN','VRSK','VZ','VRTX','V','VNO',
		'VMC','WMT','WBA','DIS','WM','WAT','WEC','WFC','WDC','WU','WY','WHR','WMB',
		'WLTW','WYNN','XEL','XRX','XLNX','XL','XYL','YUM','ZBH','ZION','ZTS']

stocks = pd.DataFrame()
x = pd.read_csv('Data/Stocks/CI_data.csv').drop(['Open','High','Low','Close','Adj Close', 'Volume', 'Name'], axis = 1)
stocks['Date'] = x['Date']
for i in s_and_p:
    s = pd.read_csv('Data/Stocks/' + i + '_data.csv').drop(['Open','High','Low','Adj Close','Volume', 'Name'], axis = 1)
    apd = s[s['Date'] >= '1994-01-01'].copy()
    stocks[i] = apd['Close']

stocks.dropna(axis = 'columns', inplace = True)

stocks['libor'] = libor[' value']
stocks['cpi'] = cpiDaily
stocks['gdp'] = gdpDaily
stocks['I'] = inflationDaily
stocks['U'] = unemploymentDaily
stocks.head()

Loading Stock Data...


,Date,MMM,ABT,ATVI,ADBE,AMD,AES,AFL,APD,ALK,...,WMB,XEL,XRX,XLNX,ZION,libor,cpi,gdp,I,U
0,1994-01-03,26.71875,6.481007,1.166667,2.656250,8.9375,5.784790,2.270833,20.987511,3.53125,...,5.986829,21.5625,38.702240,4.052083,9.12500,3.2500,146.3,10188.954,2.8,7.3
1,1994-01-04,26.50000,6.621288,1.104167,2.703125,9.1250,5.784790,2.260417,20.756245,3.50000,...,5.986829,21.4375,38.812035,4.020833,9.12500,3.2500,146.3,10188.954,2.8,7.3
2,1994-01-05,26.43750,6.761569,1.166667,2.812500,9.0625,5.724110,2.333333,21.045328,3.68750,...,6.142736,21.5000,38.866932,3.979167,9.15625,3.2500,146.3,10188.954,2.8,7.3
3,1994-01-06,26.81250,6.677401,1.114583,3.312500,9.1250,5.744337,2.291667,21.392229,3.65625,...,6.080373,21.3750,39.800175,3.958333,9.21875,3.1875,146.3,10188.954,2.8,7.3
4,1994-01-07,27.03125,6.677401,1.093750,3.281250,9.1875,5.784790,2.239583,21.160961,3.62500,...,5.924466,21.3750,39.855072,4.072917,9.21875,3.1250,146.3,10188.954,2.8,7.3


In [5]:
#%%
data_training1 = stocks[stocks['Date']<'2009-01-01'].copy()
data_test = stocks[stocks['Date']>='2009-01-01'].copy()

data_training1 = data_training1.drop(data_training1.loc[:, 'Date':'TGT'], axis = 1)
data_training = np.array(data_training1)

#Create X,Y Train Set
X_train = np.expand_dims(data_training, axis = 2)
y_train = data_training1.drop(['libor','cpi','gdp','I','U'], axis = 1)
y_train = np.array(y_train)

#Create X test Set, y actual set
data_test = data_test.drop(data_test.loc[:,'Date':'TGT'], axis = 1)
data_test = np.array(data_test)
X_test = np.expand_dims(data_test, axis = 2)

In [8]:
data_training1

,TXN,TXT,TMO,TIF,TJX,TRV,TSN,UDR,USB,UNP,...,WMB,XEL,XRX,XLNX,ZION,libor,cpi,gdp,I,U
0,3.906250,14.46875,18.333334,3.906250,0.875000,22.093750,16.000000,13.375,3.833333,10.506250,...,5.986829,21.562500,38.702240,4.052083,9.125000,3.25000,146.300,10188.954,2.8,7.3
1,3.875000,14.50000,18.333334,3.921875,0.898438,22.125000,16.500000,13.375,3.888889,10.611313,...,5.986829,21.437500,38.812035,4.020833,9.125000,3.25000,146.300,10188.954,2.8,7.3
2,4.125000,14.75000,18.166666,3.937500,0.886719,21.875000,16.083334,13.625,3.916667,10.863462,...,6.142736,21.500000,38.866932,3.979167,9.156250,3.25000,146.300,10188.954,2.8,7.3
3,4.226562,14.68750,18.222221,3.906250,0.843750,21.625000,16.500000,13.125,3.916667,11.157638,...,6.080373,21.375000,39.800175,3.958333,9.218750,3.18750,146.300,10188.954,2.8,7.3
4,4.273438,14.75000,18.333334,3.812500,0.835938,21.781250,16.333334,13.375,3.861111,11.031563,...,5.924466,21.375000,39.855072,4.072917,9.218750,3.12500,146.300,10188.954,2.8,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773,14.450000,12.78000,32.930000,22.340000,4.950000,41.660000,8.070000,13.140,23.990000,23.045000,...,10.972592,17.830000,20.105402,16.590000,22.100000,1.87625,211.398,15328.027,1.7,7.1
3774,14.350000,12.91000,32.820000,22.420000,5.010000,42.470001,8.200000,13.390,24.250000,23.485001,...,11.192696,17.980000,20.237154,16.290001,22.309999,1.86750,211.398,15328.027,1.7,7.1
3775,14.670000,12.75000,32.750000,21.500000,4.935000,43.580002,7.850000,12.590,23.709999,23.040001,...,11.347584,18.000000,19.472990,16.570000,21.719999,1.82500,211.398,15328.027,1.7,7.1
3776,15.160000,12.89000,33.540001,23.030001,5.010000,44.930000,8.550000,13.240,24.110001,23.405001,...,11.673664,18.240000,20.368906,17.620001,22.889999,1.63500,211.398,15328.027,1.7,7.1


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import optimizers

# All parameter gradients will be clipped to
# a maximum norm of 1.


#Adjust shape as needed for data

regressior = Sequential()

regressior.add(LSTM(units = 256, activation = 'relu',  return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 128, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 128, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 64, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 35))
regressior.summary()

#opt = optimizers.SGD(lr=0.001, clipnorm = 1)
regressior.compile(optimizer='adam', loss = 'mean_absolute_error')
regressior.fit(X_train, y_train, epochs=5, batch_size=8)

#%%
#Run LSTM on Test, output predictions

y_pred = regressior.predict(X_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 256)           264192    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)  